In [4]:
import numpy as np
import tensorflow as tf
import pathlib

2022-03-31 10:21:25.843086: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-31 10:21:25.843128: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
data_dir = tf.keras.utils.get_file(
    origin="http://mireu-server.iptime.org:8000/list/HDD1/hana/Garbage_Images.tar.gz",
    fname="Garbage_Images",
    untar=True
)

img_list = pathlib.Path(data_dir)
image_count = len(list(img_list.glob('*/*.jpg')))
images_categories = ['metal', 'glass', 'paper', 'trash', 'cardboard', 'plastic']

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.1
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(512, 384),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    seed=0
)

validation_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(512, 384),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    seed=0
)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [ ]:
def ai_models():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(512, 384, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=2),

        tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2),

        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2),

        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(64, activation='relu'),

        tf.keras.layers.Dense(len(images_categories), activation='softmax')
    ])
    # model.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['acc'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [ ]:
# 새로운 모델 객체를 만들고 훈련합니다
model = ai_models()

model.fit(train_generator, validation_data=validation_generator, epochs=50)

# SavedModel로 전체 모델을 저장합니다
model.save('Data/ai_models')

In [ ]:
os.listdir(cp_callback)

In [7]:
com_models = tf.keras.models.load_model('Data/ai_models')

com_models.summary()

2022-03-31 10:21:32.869724: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-31 10:21:32.869771: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-31 10:21:32.869802: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (HOSTING): /proc/driver/nvidia/version does not exist
2022-03-31 10:21:32.870086: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 384, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 192, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 192, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 96, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 96, 32)       18464     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 64, 48, 32)       0

In [ ]:
com_models.evaluate(validation_generator, verbose=2)

In [8]:
img=tf.keras.preprocessing.image.load_img("1.png", target_size=(512, 384))

x=tf.keras.preprocessing.image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = com_models.predict(images, batch_size=10)

In [50]:
print(classes[0][5])
images_categories = ['metal', 'glass', 'paper', 'trash', 'cardboard', 'plastic']

a = 0
for images_categorie in images_categories:
    # print(classes[0][a], images_categorie)
    if classes[0][a] == 1:
        # return classes[0]
        print({images_categorie:classes[0][a]})
    a += 1

0.0
{'glass': 1.0}


In [13]:
a=np.array([2,6,1,5])

print("Array:")
print(a)

req_index=np.argmax(a)
print("\nIndex with the largest value:")
print(req_index)

print("\nThe largest value in the array:")
print(a[req_index])

Array:
[2 6 1 5]

Index with the largest value:
1

The largest value in the array:
6


In [ ]:
import matplotlib.pyplot as pyplot

pyplot.figure(figsize=(18, 6))

# 에포크별 정확도
pyplot.subplot(1,2,1)
pyplot.plot(com_models.history["acc"], label="accuracy")
pyplot.plot(com_models.history["val_acc"], label="val_accuracy")
pyplot.title("accuracy")
pyplot.legend()

# 에포크별 손실률
pyplot.subplot(1,2,2)
pyplot.plot(com_models.history["loss"], label="loss")
pyplot.plot(com_models.history["val_loss"], label="val_loss")
pyplot.title("loss")
pyplot.legend()

pyplot.show()